<a href="https://colab.research.google.com/github/datawilly/Procesamiento-de-lenguaje-natural/blob/main/PEC2_PLN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [ ]:
!python -m spacy download es_core_news_sm

import re
import spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 20.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
import spacy


In [ ]:
# Ruta del archivo CSV
ruta_archivo = "/content/drive/MyDrive/Maestría en Ciencia de Datos/PLN/ner.csv"

# Leer el archivo CSV en un DataFrame de pandas
df = pd.read_csv(ruta_archivo)

# Mostrar las primeras filas del DataFrame para verificar que se haya cargado correctamente
df.head()


,Sentence #,Sentence,POS,Tag
0,Sentence: 1,Thousands of demonstrators have marched throug...,"['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '..."
1,Sentence: 2,Families of soldiers killed in the conflict jo...,"['NNS', 'IN', 'NNS', 'VBN', 'IN', 'DT', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,Sentence: 3,They marched from the Houses of Parliament to ...,"['PRP', 'VBD', 'IN', 'DT', 'NNS', 'IN', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,Sentence: 4,"Police put the number of marchers at 10,000 wh...","['NNS', 'VBD', 'DT', 'NN', 'IN', 'NNS', 'IN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,Sentence: 5,The protest comes on the eve of the annual con...,"['DT', 'NN', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'D...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


In [ ]:
# Número total de oraciones
num_oraciones = len(df)
print(f"Número total de oraciones: {num_oraciones}")

# Número total de tokens
num_tokens = sum(len(sentence) for sentence in df['Sentence'])
print(f"Número total de tokens: {num_tokens}")

Número total de oraciones: 47959
Número total de tokens: 944700


In [ ]:
# Obtener todos los tokens únicos
all_tokens = [token for sentence in df['Sentence'] for token in sentence]
vocab = set(all_tokens)
print(f"Tamaño del vocabulario: {len(vocab)}")


Tamaño del vocabulario: 29448


In [ ]:
sentence_lengths = [len(sentence) for sentence in df['Sentence']]
avg_sentence_length = sum(sentence_lengths) / len(sentence_lengths)
print(f"Longitud promedio de las oraciones: {avg_sentence_length:.2f} tokens")

Longitud promedio de las oraciones: 19.70 tokens


In [ ]:
pos_tags = [tag for sentence in df['PoSTags'] for token, tag in sentence]
pos_counts = Counter(pos_tags)
print(pos_counts)

Counter({'NOUN': 235463, 'VERB': 122739, 'ADP': 118392, 'PROPN': 116375, 'DET': 96896, 'ADJ': 81602, 'AUX': 45404, 'PRON': 32335, 'PART': 26979, 'CCONJ': 23805, 'ADV': 20746, 'SCONJ': 13782, 'NUM': 9819, 'X': 280, 'INTJ': 79, 'PUNCT': 7, 'SYM': 1})


In [ ]:
from collections import Counter

# Crear una lista vacía para almacenar todas las etiquetas
all_tags = []

# Iterar sobre cada fila del DataFrame
for tags in df['Tag']:
    # Dividir la cadena de etiquetas por las comas
    tag_list = tags.strip('[]').split("', '")

    # Agregar cada etiqueta a la lista all_tags
    for tag in tag_list:
        all_tags.append(tag.strip("'"))

# Contar los valores únicos utilizando Counter
tag_counts = Counter(all_tags)

# Imprimir los recuentos
for tag, count in tag_counts.items():
    print(f"{tag}: {count}")

O: 887908
B-geo: 37644
B-gpe: 15870
B-per: 16990
I-geo: 7414
B-org: 20143
I-org: 16784
B-tim: 20333
B-art: 402
I-art: 297
I-per: 17251
I-gpe: 198
I-tim: 6528
B-nat: 201
B-eve: 308
I-eve: 253
I-nat: 51


In [ ]:
# Eliminar duplicados (si los hay)
df = df.drop_duplicates()

# Manejar valores nulos (si los hay)
df = df.dropna()

# Normalización de texto
df['Sentence'] = df['Sentence'].apply(lambda x: x.lower())  # Convertir a minúsculas
df['Sentence'] = df['Sentence'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))  # Eliminar puntuación y caracteres especiales

# Tokenizar las oraciones
df['Sentence'] = df['Sentence'].apply(lambda x: x.split( ))

# Convertir las listas de etiquetas en listas separadas
df['POS'] = df['POS'].apply(lambda x: x.strip('[]').split(', '))
df['Tag'] = df['Tag'].apply(lambda x: x.strip('[]').split(', '))

# Mostrar las primeras filas del DataFrame para verificar los cambios
df.head()

,Sentence #,Sentence,POS,Tag
0,Sentence: 1,"[thousands, of, demonstrators, have, marched, ...","['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '..."
1,Sentence: 2,"[families, of, soldiers, killed, in, the, conf...","['NNS', 'IN', 'NNS', 'VBN', 'IN', 'DT', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,Sentence: 3,"[they, marched, from, the, houses, of, parliam...","['PRP', 'VBD', 'IN', 'DT', 'NNS', 'IN', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,Sentence: 4,"[police, put, the, number, of, marchers, at, w...","['NNS', 'VBD', 'DT', 'NN', 'IN', 'NNS', 'IN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,Sentence: 5,"[the, protest, comes, on, the, eve, of, the, a...","['DT', 'NN', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'D...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


In [ ]:
# Cargar el modelo pre-entrenado de spaCy para el inglés
nlp = spacy.load("en_core_web_sm")

# Función para extraer palabras clave (KPE)
def extraer_palabras_clave(lista_palabras):
    texto = ' '.join(lista_palabras)
    doc = nlp(texto)
    palabras_clave = [token.text for token in doc if not token.is_stop]
    return palabras_clave

# Función para realizar PoS tagging
def pos_tagging(lista_palabras):
    texto = ' '.join(lista_palabras)
    doc = nlp(texto)
    pos_tags = [(token.text, token.pos_) for token in doc]
    return pos_tags

# Aplicar extracción de palabras clave y PoS tagging al DataFrame
df['PalabrasClave'] = df['Sentence'].apply(extraer_palabras_clave)
df['PoSTags'] = df['Sentence'].apply(pos_tagging)

# Mostrar las primeras filas del DataFrame con las nuevas columnas
df.head()



,Sentence #,Sentence,POS,Tag,PalabrasClave,PoSTags
0,Sentence: 1,"[thousands, of, demonstrators, have, marched, ...","['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '...","[thousands, demonstrators, marched, london, pr...","[(thousands, NOUN), (of, ADP), (demonstrators,..."
1,Sentence: 2,"[families, of, soldiers, killed, in, the, conf...","['NNS', 'IN', 'NNS', 'VBN', 'IN', 'DT', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","[families, soldiers, killed, conflict, joined,...","[(families, NOUN), (of, ADP), (soldiers, NOUN)..."
2,Sentence: 3,"[they, marched, from, the, houses, of, parliam...","['PRP', 'VBD', 'IN', 'DT', 'NNS', 'IN', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","[marched, houses, parliament, rally, hyde, park]","[(they, PRON), (marched, VERB), (from, ADP), (..."
3,Sentence: 4,"[police, put, the, number, of, marchers, at, w...","['NNS', 'VBD', 'DT', 'NN', 'IN', 'NNS', 'IN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","[police, number, marchers, organizers, claimed]","[(police, NOUN), (put, VERB), (the, DET), (num..."
4,Sentence: 5,"[the, protest, comes, on, the, eve, of, the, a...","['DT', 'NN', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'D...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","[protest, comes, eve, annual, conference, brit...","[(the, DET), (protest, NOUN), (comes, VERB), (..."


In [ ]:
# Cargar el modelo pre-entrenado de spaCy para el inglés
nlp = spacy.load("en_core_web_sm")

# Función para realizar el reconocimiento de entidades utilizando el modelo pre-entrenado de spaCy
def reconocer_entidades_modelo(lista_palabras):
    texto = ' '.join(lista_palabras)
    doc = nlp(texto)
    entidades = [(ent.text, ent.label_) for ent in doc.ents]
    return entidades

# Aplicar el reconocimiento de entidades al DataFrame
df['EntidadesModelo'] = df['Sentence'].apply(reconocer_entidades_modelo)

# Mostrar las primeras filas del DataFrame con las nuevas columnas
df.head()



,Sentence #,Sentence,POS,Tag,PalabrasClave,PoSTags,EntidadesModelo
0,Sentence: 1,"[thousands, of, demonstrators, have, marched, ...","['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '...","[thousands, demonstrators, marched, london, pr...","[(thousands, NOUN), (of, ADP), (demonstrators,...","[(thousands, CARDINAL), (london, GPE), (iraq, ..."
1,Sentence: 2,"[families, of, soldiers, killed, in, the, conf...","['NNS', 'IN', 'NNS', 'VBN', 'IN', 'DT', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","[families, soldiers, killed, conflict, joined,...","[(families, NOUN), (of, ADP), (soldiers, NOUN)...","[(bush, PERSON), (number one, CARDINAL)]"
2,Sentence: 3,"[they, marched, from, the, houses, of, parliam...","['PRP', 'VBD', 'IN', 'DT', 'NNS', 'IN', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","[marched, houses, parliament, rally, hyde, park]","[(they, PRON), (marched, VERB), (from, ADP), (...","[(hyde, GPE)]"
3,Sentence: 4,"[police, put, the, number, of, marchers, at, w...","['NNS', 'VBD', 'DT', 'NN', 'IN', 'NNS', 'IN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","[police, number, marchers, organizers, claimed]","[(police, NOUN), (put, VERB), (the, DET), (num...",[]
4,Sentence: 5,"[the, protest, comes, on, the, eve, of, the, a...","['DT', 'NN', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'D...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","[protest, comes, eve, annual, conference, brit...","[(the, DET), (protest, NOUN), (comes, VERB), (...","[(the eve, DATE), (annual, DATE), (britain, GP..."


In [ ]:
# Contar las entidades nombradas de cada tipo
ner_counts = df['EntidadesModelo'].apply(lambda x: Counter([label for _, label in x])).sum(axis=0)
print(ner_counts)

Counter({'GPE': 27665, 'DATE': 19123, 'NORP': 16474, 'PERSON': 11746, 'ORG': 9830, 'CARDINAL': 8454, 'ORDINAL': 1780, 'LOC': 1571, 'TIME': 942, 'EVENT': 299, 'QUANTITY': 272, 'MONEY': 257, 'FAC': 212, 'PERCENT': 147, 'PRODUCT': 44, 'LANGUAGE': 30, 'WORK_OF_ART': 22, 'LAW': 15})


In [ ]:
# Obtener las entidades nombradas más frecuentes
entity_counts = df['EntidadesModelo'].apply(lambda x: Counter([entity for entity, _ in x])).sum(axis=0)
print(entity_counts.most_common(10))

[('two', 2101), ('us', 1969), ('iraq', 1700), ('the united states', 1317), ('iran', 1275), ('tuesday', 1258), ('thursday', 1200), ('wednesday', 1137), ('friday', 1132), ('iraqi', 1118)]


In [ ]:
# Calcular la longitud promedio de las entidades nombradas
entity_lengths = [len(entity) for entities in df['EntidadesModelo'] for entity, _ in entities]
avg_entity_length = sum(entity_lengths) / len(entity_lengths)
print(f"Longitud promedio de entidades nombradas: {avg_entity_length:.2f}")

Longitud promedio de entidades nombradas: 8.97


In [ ]:
# Calcular la cobertura de entidades nombradas
total_tokens = sum(len(sentence) for sentence in df['Sentence'])
tokens_with_ner = sum(len([token for token in sentence if any(token in entity for entity, _ in entities)]) for sentence, entities in zip(df['Sentence'], df['EntidadesModelo']))
ner_coverage = tokens_with_ner / total_tokens
print(f"Cobertura de entidades nombradas: {ner_coverage:.2%}")

Cobertura de entidades nombradas: 20.91%


In [ ]:
# Filtrar el DataFrame para obtener frases con entidades de tipo ORGANIZATION
df_entidades_organizacion = df[df['EntidadesOrganizacion'].apply(len) > 0]

# Mostrar las primeras filas del DataFrame filtrado
df_entidades_organizacion.head()

,Sentence #,Sentence,POS,Tag,PalabrasClave,PoSTags,EntidadesOrganizacion
7,Sentence: 8,"[the, international, atomic, energy, agency, i...","['DT', 'NNP', 'NNP', 'NNP', 'NNP', 'VBZ', 'TO'...","['O', 'B-org', 'I-org', 'I-org', 'I-org', 'O',...","[international, atomic, energy, agency, hold, ...","[(the, DET), (international, ADJ), (atomic, AD...",[the international atomic energy agency]
11,Sentence: 12,"[the, european, union, with, us, backing, has,...","['DT', 'NNP', 'NNP', ',', 'IN', 'NNP', 'NN', '...","['O', 'B-org', 'I-org', 'O', 'O', 'B-gpe', 'O'...","[european, union, backing, threatened, refer, ...","[(the, DET), (european, PROPN), (union, PROPN)...","[the european union, the un security council]"
22,Sentence: 23,"[iraqi, military, officials, say, tanks, and, ...","['JJ', 'JJ', 'NNS', 'VBP', 'NNS', 'CC', 'NNS',...","['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '...","[iraqi, military, officials, tanks, troops, ar...","[(iraqi, ADJ), (military, ADJ), (officials, NO...",[al qaida]
25,Sentence: 26,"[officials, say, al, qaida, in, iraq, fighters...","['NNS', 'VBP', 'NNP', 'NNP', 'IN', 'NNP', 'NNS...","['O', 'O', 'B-org', 'I-org', 'I-org', 'I-org',...","[officials, al, qaida, iraq, fighters, fled, s...","[(officials, NOUN), (say, VERB), (al, PROPN), ...",[al qaida]
57,Sentence: 58,"[us, army, officials, said, wednesday, that, t...","['NNP', 'NNP', 'NNS', 'VBD', 'NNP', 'IN', 'PRP...","['B-gpe', 'B-org', 'O', 'O', 'B-tim', 'O', 'O'...","[army, officials, said, wednesday, renew, cont...","[(us, PRON), (army, NOUN), (officials, NOUN), ...","[us army, the halliburton company]"
...,...,...,...,...,...,...,...
47939,Sentence: 47940,"[the, commander, of, nato, s, afghan, force, b...","['DT', 'NN', 'IN', 'NNP', 'POS', 'JJ', 'NN', '...","['O', 'O', 'O', 'B-org', 'O', 'B-gpe', 'O', 'O...","[commander, nato, s, afghan, force, british, g...","[(the, DET), (commander, NOUN), (of, ADP), (na...",[nato]
47940,Sentence: 47941,"[referring, to, the, fight, in, southern, afgh...","['VBG', 'TO', 'DT', 'NN', 'IN', 'JJ', 'NNP', '...","['O', 'O', 'O', 'O', 'O', 'B-geo', 'I-geo', 'O...","[referring, fight, southern, afghanistan, nato...","[(referring, VERB), (to, ADP), (the, DET), (fi...","[nato, nato]"
47944,Sentence: 47945,"[thura, shwe, mahn, said, those, actions, pose...","['NNP', 'NNP', 'NNP', 'VBD', 'DT', 'NNS', 'VBD...","['B-org', 'I-org', 'I-org', 'O', 'O', 'O', 'O'...","[thura, shwe, mahn, said, actions, posed, thre...","[(thura, PROPN), (shwe, PROPN), (mahn, PROPN),...",[thura shwe mahn]
47951,Sentence: 47952,"[the, monitoring, board, is, comprised, of, af...","['DT', 'NN', 'NN', 'VBZ', 'VBN', 'IN', 'JJ', '...","['O', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', '...","[monitoring, board, comprised, afghan, interna...","[(the, DET), (monitoring, NOUN), (board, NOUN)...","[the monitoring board, the united nations]"


In [ ]:
# Cargar el modelo pre-entrenado de spaCy para el inglés
nlp = spacy.load("en_core_web_sm")

# Función para realizar el reconocimiento de entidades de tipo ORGANIZATION
def reconocer_entidades_organizacion(lista_palabras):
    texto = ' '.join(lista_palabras)
    doc = nlp(texto)
    entidades_organizacion = []
    for ent in doc.ents:
        if ent.label_ == 'ORG':
            entidades_organizacion.append(ent.text)
    return entidades_organizacion

# Aplicar el reconocimiento de entidades al DataFrame
df['EntidadesOrganizacion'] = df['Sentence'].apply(reconocer_entidades_organizacion)

# Mostrar las primeras filas del DataFrame con las nuevas columnas
df.head()


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


,Sentence #,Sentence,POS,Tag,PalabrasClave,PoSTags,EntidadesOrganizacion
0,Sentence: 1,"[thousands, of, demonstrators, have, marched, ...","['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '...","[thousands, demonstrators, marched, london, pr...","[(thousands, NOUN), (of, ADP), (demonstrators,...",[]
1,Sentence: 2,"[families, of, soldiers, killed, in, the, conf...","['NNS', 'IN', 'NNS', 'VBN', 'IN', 'DT', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","[families, soldiers, killed, conflict, joined,...","[(families, NOUN), (of, ADP), (soldiers, NOUN)...",[]
2,Sentence: 3,"[they, marched, from, the, houses, of, parliam...","['PRP', 'VBD', 'IN', 'DT', 'NNS', 'IN', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","[marched, houses, parliament, rally, hyde, park]","[(they, PRON), (marched, VERB), (from, ADP), (...",[]
3,Sentence: 4,"[police, put, the, number, of, marchers, at, w...","['NNS', 'VBD', 'DT', 'NN', 'IN', 'NNS', 'IN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","[police, number, marchers, organizers, claimed]","[(police, NOUN), (put, VERB), (the, DET), (num...",[]
4,Sentence: 5,"[the, protest, comes, on, the, eve, of, the, a...","['DT', 'NN', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'D...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","[protest, comes, eve, annual, conference, brit...","[(the, DET), (protest, NOUN), (comes, VERB), (...",[]


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from torch.utils.data import DataLoader
from datasets import load_dataset
import torch
from tqdm import tqdm

# Cargar el conjunto de datos
dataset = load_dataset("ag_news")

# Tokenizar los datos
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
tokenized_dataset = dataset.map(lambda x: tokenizer(x["text"], truncation=True, padding="max_length", max_length=512), batched=True)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [ ]:
# Dividir el conjunto de datos en entrenamiento y prueba
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset = tokenized_dataset.shuffle(seed=42)
train_dataset = tokenized_dataset["train"].select(range(5000))
test_dataset = tokenized_dataset["test"].select(range(1000))

# Crear DataLoader
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=data_collator)
test_loader = DataLoader(test_dataset, batch_size=8, collate_fn=data_collator)

# Preparar el modelo para el fine-tuning
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=4)

# Definir la función de entrenamiento
def train_loop(model, train_loader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    return total_loss / len(train_loader)

# Función de evaluación
def evaluate(model, test_loader, device):
    model.eval()
    correct_predictions = 0
    total_predictions = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            _, predicted = torch.max(outputs.logits, dim=1)
            correct_predictions += (predicted == labels).sum().item()
            total_predictions += labels.size(0)

    return correct_predictions / total_predictions

# Entrenar el modelo
num_epochs = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

for epoch in range(num_epochs):
    train_loss = train_loop(model, train_loader, optimizer, device)
    test_accuracy = evaluate(model, test_loader, device)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

# Evaluar el modelo
test_accuracy = evaluate(model, test_loader, device)
print(f"Final Test Accuracy: {test_accuracy:.4f}")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 625/625 [03:39<00:00,  2.85it/s]


Epoch 1/3, Train Loss: 0.4257, Test Accuracy: 0.9030


100%|██████████| 625/625 [03:42<00:00,  2.81it/s]


Epoch 2/3, Train Loss: 0.2164, Test Accuracy: 0.8960


100%|██████████| 625/625 [03:42<00:00,  2.81it/s]


Epoch 3/3, Train Loss: 0.1278, Test Accuracy: 0.8980
Final Test Accuracy: 0.8980


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from torch.utils.data import DataLoader
from datasets import load_dataset
import torch
from tqdm import tqdm

# Cargar el conjunto de datos
dataset = load_dataset("imdb")

# Tokenizar los datos
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokenized_dataset = dataset.map(lambda x: tokenizer(x["text"], truncation=True, padding="max_length", max_length=512), batched=True)

# Dividir el conjunto de datos en entrenamiento y prueba
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset = tokenized_dataset.shuffle(seed=42)
train_dataset = tokenized_dataset["train"].select(range(20000))
test_dataset = tokenized_dataset["test"].select(range(2000))

# Crear DataLoader
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=data_collator)
test_loader = DataLoader(test_dataset, batch_size=8, collate_fn=data_collator)

# Preparar el modelo para el fine-tuning
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Definir la función de entrenamiento
def train_loop(model, train_loader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    return total_loss / len(train_loader)

# Función de evaluación
def evaluate(model, test_loader, device):
    model.eval()
    correct_predictions = 0
    total_predictions = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            _, predicted = torch.max(outputs.logits, dim=1)
            correct_predictions += (predicted == labels).sum().item()
            total_predictions += labels.size(0)

    return correct_predictions / total_predictions

# Entrenar el modelo
num_epochs = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

for epoch in range(num_epochs):
    train_loss = train_loop(model, train_loader, optimizer, device)
    test_accuracy = evaluate(model, test_loader, device)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

# Evaluar el modelo
test_accuracy = evaluate(model, test_loader, device)
print(f"Final Test Accuracy: {test_accuracy:.4f}")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

Epoch 1/3, Train Loss: 0.2465, Test Accuracy: 0.9310


100%|██████████| 2500/2500 [30:15<00:00,  1.38it/s]


Epoch 2/3, Train Loss: 0.1245, Test Accuracy: 0.9255


100%|██████████| 2500/2500 [30:15<00:00,  1.38it/s]


Epoch 3/3, Train Loss: 0.0614, Test Accuracy: 0.9290
Final Test Accuracy: 0.9290
